In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/852k-used-car-listings/tc20171021.csv
/kaggle/input/852k-used-car-listings/true_car_listings.csv


we have used true_car_listings.csv as testing data set
and tc20171021.csv as training data


importing libraries

In [3]:
import pandas as pd
import numpy as np
import scipy as spy
import copy
import math
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn import linear_model

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
!ls /kaggle/input/852k-used-car-listings


tc20171021.csv	true_car_listings.csv


DATA INGESTION

In [6]:
data = pd.read_csv("/kaggle/input/852k-used-car-listings/tc20171021.csv.csv")

PREVIEW THE DATASET

In [7]:
# Preview the dataset
# View the first 5, last 5 and random 10 rows
print('First five rows', '--'*55)
display(data.head())

print('Last five rows', '--'*55)
display(data.tail())

print('Random ten rows', '--'*55)
np.random.seed(1)
display(data.sample(n=10))

First five rows --------------------------------------------------------------------------------------------------------------


,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic


Last five rows --------------------------------------------------------------------------------------------------------------


,Price,Year,Mileage,City,State,Vin,Make,Model
852117,63215,2017,9,Culver City,CA,YV1A22MK9H1013237,Volvo,S90T6
852118,72260,2017,3201,Englewood,NJ,YV4A22PL3H1186162,Volvo,XC90T6
852119,55999,2016,28941,Fort Collins,CO,YV4A22PL4G1000868,Volvo,XC90AWD
852120,60240,2017,3005,San Leandro,CA,YV4A22NLXH1006162,Volvo,V90
852121,76995,2017,2502,New York,NY,YV4BC0ZX1H1109845,Volvo,XC90T8


Random ten rows --------------------------------------------------------------------------------------------------------------


,Price,Year,Mileage,City,State,Vin,Make,Model
771139,11847,2013,77350,Leesburg,FL,JTDKN3DU9D5596405,Toyota,PriusTwo
43532,24995,2012,54656,Teterboro,NJ,5UXZV4C50CL755416,BMW,X5AWD
275419,22011,2015,28584,Wichita,KS,1FMCU9J91FUA83065,Ford,Escape4WD
343223,45990,2017,22347,Asheville,NC,1FT7W2BT4HEC65230,Ford,Super
474821,33977,2017,11223,North Plainfield,NJ,JN8CS1MW0HM411533,INFINITI,QX70AWD
605291,32750,2015,35373,Little Rock,AR,WDCGG8JB4FG399594,Mercedes-Benz,GLK-ClassGLK350
9520,37459,2016,16874,Dallas,TX,5FRYD3H27GB017113,Acura,MDXFWD
568250,34443,2015,37428,Watertown,MA,2T2BK1BA0FC282262,Lexus,RX
823822,18251,2015,15479,Columbus,OH,3VWYT7AU8FM039300,Volkswagen,Golf
729207,16591,2016,39335,Norwood,MA,JF1GPAA68G8262241,Subaru,Impreza


**Observations:**
* The **S.No.** is simply a row identifier which can be removed later.
* The **Name** is long string comprising the Brand, Model and addition specification. These can be split to further analyse by make and model.
* The **Location** is a geographical location. We can possibly create a heat map of car make by location.
* The **Fuel_Type**, **Transmission** and **Owner_Type** are categorical variable which can be possibly be onehot encoded.
* **Mileage** is a string leading with number and unit. This shall be split into the mileage number and mileage unit. Further analysis will determine if the mileage number to be converted based on the units.
* **Engine** and **Power** both are strings leading with number and unit. This shall be split and numerical portion will be used in analysis.
* **Seats** is a discrete numerical variable.
* **New_Price** has many missing values. Where it contains a value the currency term shall be removed by splitting.
* **Price** has missing values

For ease of programming analysis the dataframe column names shall be converted to lower case.

In [8]:
data.columns = [col.lower() for col in data.columns]

Let's check the first row in the dataframe to verify the change

In [9]:
data.head(1)

,price,year,mileage,city,state,vin,make,model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed


## Variable List

In [10]:
# Display list of variables in dataset
variable_list = data.columns.tolist()
print(variable_list)

['price', 'year', 'mileage', 'city', 'state', 'vin', 'make', 'model']


## Get the shape of the dataset

In [11]:
shape = data.shape
n_rows = shape[0]
n_cols = shape[1]
print(f"The Dataframe consists of '{n_rows}' rows and '{n_cols}' columns")

The Dataframe consists of '852122' rows and '8' columns


## Data info

In [12]:
# Get info of the dataframe columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852122 entries, 0 to 852121
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   price    852122 non-null  int64 
 1   year     852122 non-null  int64 
 2   mileage  852122 non-null  int64 
 3   city     852122 non-null  object
 4   state    852122 non-null  object
 5   vin      852122 non-null  object
 6   make     852122 non-null  object
 7   model    852122 non-null  object
dtypes: int64(3), object(5)
memory usage: 52.0+ MB


**Panda Object Variable states**

In [13]:
# Panda Object Variable states function

def pandas_object_states(data):
    """
    This function checks if the variable type is pandas Object and
    displays the states and counts of each
    """
    # Loop through all variables
    for var in data.columns:
        # Check for pandas Object type
        if data[var].dtypes == "object":
            print('Unique values in', var, 'are :')
            print(data[var].value_counts().sort_index())
            print('--'*55)

In [14]:
# Check the states of all pandas Object variables
pandas_object_states(data)

Unique values in city are :
city
AKRON            9
ALEXANDRIA      70
AURORA          49
AUSTIN         349
AVON             6
              ... 
kansas city     85
marietta        41
phoenix         85
salt lake       21
victorville     68
Name: count, Length: 2553, dtype: int64
--------------------------------------------------------------------------------------------------------------
Unique values in state are :
state
AK     1663
AL    13293
AR     6952
AZ    22401
Az       63
CA    80437
CO    23004
CT    10906
Ca       31
DC       25
DE     2418
FL    72530
Fl      783
GA    39414
Ga      109
HI     2995
IA     4840
ID     3501
IL    38680
IN    16465
KS     7894
KY    13691
LA     8528
MA    17990
MD    17572
ME     1920
MI    10328
MN    10996
MO    16165
MS     6242
MT     1984
Md      135
NC    37530
ND     1092
NE     4924
NH     5694
NJ    27852
NM     4729
NV     6887
NY    27932
OH    23259
OK     9584
OR    10352
Oh       52
PA    28130
RI     1858
SC     9790
SD     1

**Missing value summary function**

In [15]:
def missing_val_chk(data):
    """
    This function to checks for missing values 
    and generates a summary.
    """
    if data.isnull().sum().any() == True:
        # Number of missing in each column
        missing_vals = pd.DataFrame(data.isnull().sum().sort_values(
            ascending=False)).rename(columns={0: '# missing'})

        # Create a percentage missing
        missing_vals['percent'] = ((missing_vals['# missing'] / len(data)) *
                                   100).round(decimals=3)

        # Remove rows with 0
        missing_vals = missing_vals[missing_vals['# missing'] != 0].dropna()

        # display missing value dataframe
        print("The missing values summary")
        display(missing_vals)
    else:
        print("There are NO missing values in the dataset")

In [16]:
#Applying the missing value summary function
missing_val_chk(data)

There are NO missing values in the dataset


**Numerical type Summary**

In [17]:
# Five point summary of all numerical type variables in the dataset
data.describe().T

,count,mean,std,min,25%,50%,75%,max
price,852122.0,21464.100210,13596.202241,1500.0,13000.0,18500.0,26995.0,499500.0
year,852122.0,2013.289145,3.414987,1997.0,2012.0,2014.0,2016.0,2018.0
mileage,852122.0,52507.794921,41988.962042,5.0,23836.0,40256.0,72186.0,2856196.0


**Categorical type Summary**

In [18]:
data.describe(include=['object']).T

,count,unique,top,freq
city,852122,2553,Houston,12703
state,852122,59,TX,94654
vin,852122,852075,2G1FA1E37E9293949,2
make,852122,58,Ford,110432
model,852122,2736,Silverado,24932


**Check if column `s.no.` simliar to index**

In [19]:
# Extract the index values and convert to Pandas Series for comparison
idx = pd.Series(data.index.values, name="index")

# Extract the first column safely (whatever its name is)
first_col = data.iloc[:, 0]

# Display both to verify
print("Index values:\n", idx.head())
print("\nFirst column:\n", first_col.head())


Index values:
 0    0
1    1
2    2
3    3
4    4
Name: index, dtype: int64

First column:
 0     8995
1    10888
2     8995
3    10999
4    14799
Name: price, dtype: int64


**Compare Series idx and first_col**

In [20]:
first_col.equals(idx)

False

Therefore, let's drop the `s.no.` column

In [21]:
data.drop(columns=[col for col in ['s.no.', 'Id'] if col in data.columns], inplace=True)


Let's do a check to see if **s.no.** is in the dropped dataframe

In [22]:
data = data.rename(columns={'mileage_no': 'mileage'})

In [23]:
if data.mileage.dtypes != np.number:
    # This will convert all numeric strings to float.
    # If a value is non numeric it will set to NaN
    data['mileage'] = pd.to_numeric(data.mileage, errors='coerce')

In [ ]:

if 'engine' in data.columns:
    data[['engine_disp', 'eng_disp_unit']] = data['engine'].str.split(' ', n=1, expand=True)
else:
    print("⚠️ No 'engine' column found in this dataset. Skipping engine split.")


⚠️ No 'engine' column found in this dataset. Skipping engine split.


In [25]:
data.head(1)

,price,year,mileage,city,state,vin,make,model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed


In [26]:
if 'name' in data.columns:
    data = data.drop(columns='name')


In [27]:
print(data.columns)

Index(['price', 'year', 'mileage', 'city', 'state', 'vin', 'make', 'model'], dtype='object')


In [28]:
data['car_make'].value_counts() if 'car_make' in data.columns else None


# Separating Independent and Dependent variables

In [29]:
# Separating Independent and Dependent variables

# Independent variables
X = data.drop(['price'], axis=1)

# Dependent variable
y = data[['price']]


This code cell prepares the used car data by cleaning and using Ordinal Encoding on categorical features (state, city, make, model). It then trains a Linear Regression model to predict the car's price, evaluates the model's accuracy using R² and RMSE, and determines the feature importance based on the model's coefficients.

In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# ----------------------------
# Load training and test datasets
# ----------------------------
train_data =data
test_data = pd.read_csv("/kaggle/input/852k-used-car-listings/true_car_listings.csv",
                        on_bad_lines='skip')

# Standardize column names
train_data.columns = train_data.columns.str.lower().str.strip()
test_data.columns = test_data.columns.str.lower().str.strip()

# ----------------------------
# Select important columns
# ----------------------------
important_cols = ['price', 'year', 'mileage', 'state', 'city', 'make', 'model']

X_train_selected = train_data[important_cols].copy()
X_test_selected = test_data[important_cols].copy()

# ----------------------------
# Encode categorical columns with OrdinalEncoder
# ----------------------------
categorical_cols = ['state', 'city', 'make', 'model']
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_selected[categorical_cols] = encoder.fit_transform(
    X_train_selected[categorical_cols].astype(str)
)
X_test_selected[categorical_cols] = encoder.transform(
    X_test_selected[categorical_cols].astype(str)
)

# ----------------------------
# Prepare target
# ----------------------------
Y_train = pd.to_numeric(X_train_selected['price'], errors='coerce')
Y_test = pd.to_numeric(X_test_selected['price'], errors='coerce')

# Drop rows with missing targets
train_valid_idx = ~np.isnan(Y_train)
X_train_final = X_train_selected.loc[train_valid_idx].drop(columns='price')
Y_train_final = Y_train[train_valid_idx]

test_valid_idx = ~np.isnan(Y_test)
X_test_final = X_test_selected.loc[test_valid_idx].drop(columns='price')
Y_test_final = Y_test[test_valid_idx]

print("Training shape:", X_train_final.shape)
print("Testing shape:", X_test_final.shape)

# ----------------------------
# Fit Linear Regression
# ----------------------------
model = LinearRegression(fit_intercept=True)
model.fit(X_train_final, Y_train_final)

# ----------------------------
# Evaluate on test data
# ----------------------------
Y_pred_test = model.predict(X_test_final)

r2 = r2_score(Y_test_final, Y_pred_test)
mse = mean_squared_error(Y_test_final, Y_pred_test)
rmse = np.sqrt(mse)

print("\nTest Performance:")
print("R² Score:", r2)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)

# ----------------------------
# Feature importance
# ----------------------------
coef_df = pd.DataFrame({
    'Feature': X_train_final.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

print("\nTop Features by Influence:")
print(coef_df)


Training shape: (852122, 6)
Testing shape: (852122, 6)

Test Performance:
R² Score: 0.20561683789783713
Mean Squared Error: 146846889.75734496
Root Mean Squared Error: 12118.039847984697

Top Features by Influence:
   Feature  Coefficient
0     year   742.559497
4     make   -66.413235
2    state    22.884413
5    model     1.360293
3     city    -0.304090
1  mileage    -0.092401
